##  Uso del método de Leonhard Euler para el cálculo de trayectorias de reentrada
Sí, el uso de métodos numéricos desarrollados por Leonhard Euler, incluido el enfoque de **Euler Modificado** (o una forma de aproximación de pasos pequeños) para calcular la trayectoria de reentrada de John Glenn, fue un hecho basado en la vida real de **Katherine Johnson** en la **NASA**.<br>
Aunque métodos más avanzados como Runge-Kutta existían, la decisión de utilizar el método de Euler en ese momento crucial se debe a varios factores técnicos e históricos: <br>

### ¿Por qué usaron el Método de Euler?
- Rapidez y Simplicidad Manual: El método de Euler es un algoritmo de primer orden que permite calcular aproximaciones de ecuaciones diferenciales paso a paso. Aunque menos preciso que Runge-Kutta, es mucho más sencillo y rápido de ejecutar a mano o con calculadoras mecánicas. Cuando el ordenador IBM 7090 fallaba o necesitaba verificación inmediata, Katherine necesitaba una solución rápida y "a mano".
- "La vieja matemática es la mejor": En la película, este método es descrito como "vieja matemática" (old math). En esa época, se confiaba más en la intuición y los cálculos verificados manualmente por humanos que en las nuevas, y a menudo inestables, computadoras.
- Solución a un problema de inestabilidad: Durante la reentrada, los ingenieros de la NASA se enfrentaban a ecuaciones diferenciales no lineales y muy inestables. Las aproximaciones de Euler permitían "trocear" la curva compleja de la trayectoria en pequeños segmentos lineales, lo cual era suficiente y más seguro para encontrar el punto de aterrizaje correcto.<br> 
### ¿Por qué no usaron Runge-Kutta?
Si bien **Runge-Kutta** es un método más preciso (de orden superior), para la situación de la película:<br> 
- **Era complejo para el cálculo manual:** Ejecutar Runge-Kutta de cuarto orden (RK4) a mano paso a paso hubiera sido extremadamente laborioso y propenso a errores humanos.<br>
- **La confianza en la "Computadora Humana":** John Glenn exigió específicamente que **Katherine Johnson**, en quien confiaba más que en la máquina **IBM**, verificara los números a mano antes de su vuelo. Ella usó las herramientas que mejor dominaba para obtener resultados rápidos y fiables.<br>
  
En resumen, el método de Euler fue utilizado no porque fuera el más avanzado, sino porque era un **método robusto, directo y, sobre todo, "manuable"**, que permitió obtener una solución fiable a tiempo para la misión de John Glenn.<br>






### CELDA 1: Parámetros Globales

Aquí se definen todas las constantes físicas y de configuración:
| Variable         | Significado                   | Valor         |
| ---------------- | ----------------------------- | ------------- |
| `g`              | Gravedad                      | 0.4 px/frame² |
| `v0`             | Velocidad inicial             | 15 px/frame   |
| `angle`          | Ángulo de lanzamiento         | 45°           |
| `numProjectiles` | Cantidad de proyectiles       | 5             |
| `trailLength`    | Longitud de la estela visual  | 50 puntos     |
| `restitution`    | Pérdida de energía en rebotes | 70% (0.7)     |
| `dt`             | Paso de tiempo                | 1 frame       |

**Nota clave:** Se usa var en lugar de let para que las variables sean globales y accesibles entre todas las celdas de JupyterLite.

In [1]:
// ============================================================
// 5.2 Celda 1: Parámetros Globales
// PARÁMETROS FÍSICOS Y DE SIMULACIÓN
// ============================================================

var g = 0.4;              // Aceleración de la gravedad [pixels/frame²]
var v0 = 15;              // Velocidad inicial de lanzamiento [pixels/frame]
var angle = 45;           // Ángulo de lanzamiento [grados]
var numProjectiles = 5;   // Número de proyectiles simultáneos
var trailLength = 50;     // Longitud máxima de la estela de rastro
var projectiles;          // Arreglo de objetos proyectil (inicializado en setup)
var restitution = 0.7;    // Coeficiente de restitución para rebotes (0-1)
var dt = 1;               // Paso de tiempo Δt (1 frame = unidad temporal)
// ============================================================
// Nota importante: Las variables deben declararse con var (no let) 
// para tener alcance global entre celdas en JupyterLite. No inicializar objetos 
// que dependan de height o width aquí, ya que el canvas aún no existe.

### CELDA 2: Inicialización (setup)

Esta función se ejecuta una sola vez al inicio:
- 1 Crea el canvas del tamaño de la ventana del navegador
- 2 Configura modos: grados para ángulos, HSB para colores vibrantes
- 3 Crea los proyectiles con una dispersión angular de 5° entre cada uno

Cada proyectil es un objeto JavaScript con propiedades:
- x, y: posición actual
- vx, vy: velocidades en cada eje
- trail: arreglo que guarda posiciones anteriores (para el rastro visual)
- active: indica si está en movimiento
- hue: color único (0, 60, 120, 180, 240)
- radius: tamaño del círculo (12, 11, 10, 9, 8)

In [2]:
// ============================================================
// SETUP: INICIALIZACIÓN DEL CANVAS Y OBJETOS FÍSICOS
// 5.3 Celda 2: Función setup()
// ============================================================

function setup() {
  // Crear canvas de tamaño completo de la ventana del navegador
  createCanvas(innerWidth, innerHeight);
  
  // Configurar modos de ángulo y color
  angleMode(DEGREES);     // Trabajar en grados para el ángulo de lanzamiento
  colorMode(HSB);         // Hue-Saturation-Brightness para colores vibrantes
  
  // Inicializar arreglo de proyectiles vacío
  projectiles = [];
  
  // Crear cada proyectil con ángulo ligeramente diferente (dispersión de 5°)
  for (let i = 0; i < numProjectiles; i++) {
    let launchAngle = angle + i * 5;
    
    projectiles.push({
      x: 50,                                    // Posición inicial X
      y: height - 40,                           // Posición inicial Y (sobre el suelo)
      vx: v0 * cos(launchAngle),                // Velocidad inicial en X
      vy: -v0 * sin(launchAngle),               // Velocidad inicial en Y (negativa = arriba)
      trail: [],                                // Arreglo para almacenar posiciones anteriores
      active: true,                             // Estado del proyectil (activo/inactivo)
      hue: i * 60,                              // Tono de color único para cada proyectil
      radius: 12 - i                            // Radio del círculo (decreciente)
    });
  }
}

In [3]:
// ============================================================
// SETUP: INICIALIZACIÓN DEL CANVAS Y OBJETOS FÍSICOS
// 5.3 Celda 2: Función setup()
// ============================================================

function setup() {
  // Crear canvas de tamaño completo de la ventana del navegador
  createCanvas(innerWidth, innerHeight);
  
  // Configurar modos de ángulo y color
  angleMode(DEGREES);     // Trabajar en grados para el ángulo de lanzamiento
  colorMode(HSB);         // Hue-Saturation-Brightness para colores vibrantes
  
  // Inicializar arreglo de proyectiles vacío
  projectiles = [];
  
  // Crear cada proyectil con ángulo ligeramente diferente (dispersión de 5°)
  for (let i = 0; i < numProjectiles; i++) {
    let launchAngle = angle + i * 5;
    
    projectiles.push({
      x: 50,                                    // Posición inicial X
      y: height - 40,                           // Posición inicial Y (sobre el suelo)
      vx: v0 * cos(launchAngle),                // Velocidad inicial en X
      vy: -v0 * sin(launchAngle),               // Velocidad inicial en Y (negativa = arriba)
      trail: [],                                // Arreglo para almacenar posiciones anteriores
      active: true,                             // Estado del proyectil (activo/inactivo)
      hue: i * 60,                              // Tono de color único para cada proyectil
      radius: 12 - i                            // Radio del círculo (decreciente)
    });
  }
}

### CELDA 3: Función de Derivadas
Define las ecuaciones diferenciales del movimiento parabólico:
- $\frac{dx}{dt}=v_x \Rightarrow$   derivada de la posición de X
- $\frac{dy}{dt}=v_y \Rightarrow$   derivada de la posición de y
- $\frac{dv_x}{dt}= 0 \Rightarrow$ sin aceleración horizontal
- $\frac{dv_y}{dt}= g \Rightarrow$ gravedad hacia abajo

Esta función recibe el estado actual del proyectil y devuelve las tasas de cambio (derivadas) de cada variable.

In [4]:
// ============================================================
// DERIVADAS DEL SISTEMA DINÁMICO
// 5.4 Celda 3: Función de Derivadas
// ============================================================
// 
// El movimiento parabólico se describe por el sistema de EDOs:
//
//   dx/dt = vx          (velocidad en x)
//   dy/dt = vy          (velocidad en y)
//   dvx/dt = 0          (aceleración en x = 0, sin resistencia del aire)
//   dvy/dt = g          (aceleración en y = gravedad)
//
// Vector de estado: y = [x, y, vx, vy]
// Función derivada: f(t, y) = [vx, vy, 0, g]

function derivatives(state) {
  return {
    dx: state.vx,    // Derivada de x es la velocidad en x
    dy: state.vy,    // Derivada de y es la velocidad en y
    dvx: 0,          // No hay aceleración horizontal (sin fricción)
    dvy: g           // Aceleración vertical = gravedad (hacia abajo)
  };
}

### CELDA 4: El Método de Euler (NÚCLEO DEL CÓDIGO)
Aquí ocurre la integración numérica. El método de Euler es el más simple:<br>
Fórmula fundamental:<br>
$y_n+1 = y_n + \Delta t \cdot f(t_n,y_n)$<br>

¿Qué significa esto?
- $y_n$: valor actual(posición o velocidad)
- $f(t_n,y_n)$: derivada evaluada en el estado actual(la pendiente)
- $\Delta t$: paso de tiempo
- $y_{n+1}$: valor en el siguiente instante 
Paso a paso en el código:<br>
- 1. Capturar estado actual: posición y velocidad del proyectil
- 2. Calcular derivadas: llamar a derivatives(state) una sola vez
- 3. Actualizar cada variable multiplicando la derivada por dt:
| Variable    | Actualización            |
| ----------- | ------------------------ |
| Posición X  | `p.x += dt * deriv.dx`   |
| Posición Y  | `p.y += dt * deriv.dy`   |
| Velocidad X | `p.vx += dt * deriv.dvx` |
| Velocidad Y | `p.vy += dt * deriv.dvy` |

Limitaciones de Euler (documentadas en los comentarios):
| Propiedad    | Valor           | Implicación                                          |
| ------------ | --------------- | ---------------------------------------------------- |
| Error local  | $O(\Delta t^2)$ | Error en cada paso proporcional al cuadrado del paso |
| Error global | $O(\Delta t)$   | Error acumulado proporcional al paso                 |
| Estabilidad  | Condicional     | Puede volverse inestable con pasos grandes           |
| Costo        | 1 evaluación    | Muy rápido pero impreciso                            |

Comparación visual: Con Δt=1 , **Euler acumula error rápidamente**. La trayectoria real debería ser una parábola perfecta, pero Euler la "escapa" ligeramente hacia abajo.

In [5]:
// ============================================================
// MÉTODO DE EULER EXPLÍCITO (EULER HACIA ADELANTE)
// 5.5 Celda 4: Implementación de Euler
// ============================================================
//
// FÓRMULA GENERAL:
//   y_{n+1} = y_n + Δt * f(t_n, y_n)
//
// INTERPRETACIÓN GEOMÉTRICA:
//   - Evalúa la pendiente al inicio del intervalo (t_n, y_n)
//   - Avanza en línea recta con esa pendiente durante Δt
//   - Es equivalente a una aproximación de primer orden (recta tangente)
//
// COMPARACIÓN CON RK4:
//   - Euler: 1 evaluación de derivada por paso
//   - RK4: 4 evaluaciones de derivada por paso
//   - Euler es más rápido pero mucho menos preciso

function eulerStep(p) {
  
  // -----------------------------------------------------------
  // ESTADO ACTUAL DEL PROYECTIL
  // Estado vector: y_n = [x, y, vx, vy] en el tiempo t_n
  // -----------------------------------------------------------
  let state = {
    x: p.x,     // Posición horizontal actual
    y: p.y,     // Posición vertical actual
    vx: p.vx,   // Velocidad horizontal actual
    vy: p.vy    // Velocidad vertical actual
  };
  
  // ===========================================================
  // CÁLCULO DE LAS DERIVADAS EN EL ESTADO ACTUAL
  // ===========================================================
  // f(t_n, y_n) = derivadas evaluadas solo al inicio del intervalo
  //
  // A diferencia de RK4 que usa múltiples puntos, Euler usa solo
  // la pendiente inicial para todo el paso
  
  let deriv = derivatives(state);
  
  // ===========================================================
  // ACTUALIZACIÓN POR EULER EXPLÍCITO
  // ===========================================================
  // y_{n+1} = y_n + Δt * f(t_n, y_n)
  //
  // Cada variable se actualiza usando la derivada correspondiente:
  
  // Actualizar posición X: x_{n+1} = x_n + Δt * vx
  p.x += dt * deriv.dx;
  
  // Actualizar posición Y: y_{n+1} = y_n + Δt * vy
  p.y += dt * deriv.dy;
  
  // Actualizar velocidad X: vx_{n+1} = vx_n + Δt * ax
  p.vx += dt * deriv.dvx;
  
  // Actualizar velocidad Y: vy_{n+1} = vy_n + Δt * ay
  p.vy += dt * deriv.dvy;
  
  // ===========================================================
  // PROPIEDADES DEL MÉTODO DE EULER:
  // - Error local por paso: O(Δt²)
  // - Error global acumulado: O(Δt)
  // - Estabilidad: Condicional (puede inestabilizarse con pasos grandes)
  // - Costo: 1 evaluación de la función derivada por paso
  // - Precisión: Baja comparada con RK4 (error ~100x mayor para Δt=0.1)
  // ===========================================================
}

### CELDA 5: Dibujo del Suelo
Dibuja una franja verde en la parte inferior (height - 40) con:
- Rectángulo de fondo
- Línea superior más oscura (borde)
- Marcas cada 100 píxeles para referencia de distancia

In [6]:
// ============================================================
// DIBUJO DEL SUELO CON REBOTE
// 5.6 Celda 5: Dibujo del Suelo
// ============================================================

function drawGround() {
  // Fondo del suelo (rectángulo verde oscuro)
  noStroke();
  fill(60, 80, 40);  // HSB: tono verde, saturación 80%, brillo 40%
  rect(0, height - 40, width, 40);
  
  // Línea superior del suelo (borde más oscuro)
  stroke(120, 60, 30);
  strokeWeight(3);
  line(0, height - 40, width, height - 40);
  
  // Marcas de distancia cada 100 píxeles (referencia visual)
  stroke(120, 40, 50);
  strokeWeight(1);
  for (let x = 0; x < width; x += 100) {
    line(x, height - 40, x, height);
  }
}

### CELDA 6: Dibujo del Cañón
Usa transformaciones geométricas de p5.js:
- push(): guarda el sistema de coordenadas actual
- translate(): mueve el origen a la base del cañón
- rotate(): inclina según el ángulo de lanzamiento
- rect(): dibuja el tubo del cañón
- pop(): restaura el sistema de coordenadas
- ellipse(): dibuja la base circular


In [7]:
// ============================================================
// DIBUJO DEL CAÑÓN DE LANZAMIENTO
// 5.7 Celda 6: Dibujo del Cañón
// ============================================================

function drawCannon() {
  push();  // Guardar sistema de coordenadas actual
  
  // Posicionar en la base del cañón (sobre el suelo)
  translate(50, height - 40);
  
  // Rotar según el ángulo de lanzamiento (negativo = hacia arriba)
  rotate(-angle);
  
  // Dibujar el tubo del cañón
  fill(0, 0, 30);  // Gris oscuro
  noStroke();
  rect(0, 0, 60, 20);  // Rectángulo de 60x20 píxeles
  
  // Restaurar sistema de coordenadas
  pop();
  
  // Dibujar la base circular del cañón
  fill(0, 0, 20);  // Gris más oscuro
  ellipse(50, height - 40, 40, 40);  // Círculo de 40px de diámetro
}

### CELDA 7: Actualización de Proyectiles
Bucle principal que procesa cada proyectil activo:<br>
Secuencia de operaciones:
- 1. Guardar posición en la estela (trail.push)
- 2. Limitar tamaño de la estela (eliminar puntos antiguos con shift())
- 3. LLAMAR A EULER: eulerStep(p) ← aquí se mueve el proyectil
- 4. Detectar colisión con el suelo:
     - Si p.y + p.radius > height - 40 (tocó el suelo):
       - Corregir posición para que no se hunda
       - Invertir velocidad Y con pérdida de energía: p.vy *= -restitution
       - Aplicar fricción horizontal: p.vx *= 0.95
       - Desactivar si la energía es muy baja
- 5. Detectar colisión con paredes: invertir velocidad X

In [8]:
// ============================================================
// ACTUALIZACIÓN DE PROYECTILES USANDO EULER
// 5.8 Celda 7: Actualización de Proyectiles
// ============================================================

function updateProjectiles() {
  
  // Iterar sobre todos los proyectiles
  for (let i = 0; i < projectiles.length; i++) {
    let p = projectiles[i];
    
    // Solo procesar proyectiles activos
    if (p.active) {
      
      // Guardar posición en la estela (rastro visual)
      p.trail.push({x: p.x, y: p.y});
      
      // Limitar longitud de la estela (eliminar puntos antiguos)
      if (p.trail.length > trailLength) {
        p.trail.shift();
      }
      
      // INTEGRACIÓN NUMÉRICA POR EULER EXPLÍCITO
      // Avanzar un paso de tiempo usando el método de Euler
      // NOTA: Reemplazamos rk4Step(p) por eulerStep(p)
      eulerStep(p);
      
      // DETECCIÓN Y RESPUESTA DE COLISIONES
      
      // Colisión con el suelo
      if (p.y + p.radius > height - 40) {
        
        // Corregir posición (evitar que se hunda en el suelo)
        p.y = height - 40 - p.radius;
        
        // Aplicar rebote: invertir velocidad vertical con pérdida de energía
        // v_after = -restitution * v_before
        p.vy *= -restitution;
        
        // Fricción con el suelo: reducir velocidad horizontal
        p.vx *= 0.95;
        
        // Desactivar proyectil si la energía es muy baja
        if (abs(p.vy) < 1 && abs(p.vx) < 0.5) {
          p.active = false;
        }
      }
      
      // Colisión con paredes laterales
      if (p.x - p.radius < 0 || p.x + p.radius > width) {
        p.vx *= -restitution;
        p.x = constrain(p.x, p.radius, width - p.radius);
      }
    }
  }
}

### CELDA 8: Renderizado Visual
Dibuja cada proyectil con efectos visuales:
| Elemento | Descripción                                                             |
| -------- | ----------------------------------------------------------------------- |
| Estela   | Puntos con transparencia decreciente (más antiguos = más transparentes) |
| Sombra   | Círculo negro desplazado (2, 2) con opacidad 50%                        |
| Cuerpo   | Círculo principal con color HSB único                                   |
| Brillo   | Pequeño círculo claro para efecto 3D                                    |



In [9]:
// ============================================================
// RENDERIZADO DE PROYECTILES (CÍRCULOS CON EFECTOS VISUALES)
// 5.9 Celda 8: Renderizado de Proyectiles
// ============================================================

function drawProjectiles() {
  
  for (let i = 0; i < projectiles.length; i++) {
    let p = projectiles[i];
    
    // Dibujar estela (rastro de trayectoria)
    noFill();
    strokeWeight(2);
    
    for (let j = 0; j < p.trail.length; j++) {
      // Calcular transparencia: puntos más recientes = más opacos
      let alpha = map(j, 0, p.trail.length, 0, 255);
      
      stroke(p.hue, 80, 100, alpha);
      let t = p.trail[j];
      point(t.x, t.y);
    }
    
    // Dibujar proyectil (círculo con sombra y brillo)
    push();
    translate(p.x, p.y);
    
    // Sombra: círculo negro desplazado
    noStroke();
    fill(0, 0, 0, 50);
    ellipse(2, 2, p.radius * 2, p.radius * 2);
    
    // Cuerpo principal: círculo de color
    fill(p.hue, 80, 100);
    ellipse(0, 0, p.radius * 2, p.radius * 2);
    
    // Brillo: pequeño círculo claro
    fill(p.hue, 40, 100);
    ellipse(-p.radius/3, -p.radius/3, p.radius, p.radius);
    
    pop();
  }
}

### CELDA 9: Reinicio
Restaura todos los proyectiles a su estado inicial:
- Posición: (50, height - 40) (boca del cañón)
- Velocidades recalculadas con ángulo de dispersión
- Estela vaciada
- Estado reactivado (active = true)<br>

Se ejecuta automáticamente cada 4 segundos (240 frames).


In [10]:
// ============================================================
// REINICIO DE PROYECTILES (NUEVA RONDA DE LANZAMIENTO)
// 5.10 Celda 9: Reinicio de Simulación
// ============================================================

function resetProjectiles() {
  
  for (let i = 0; i < numProjectiles; i++) {
    let p = projectiles[i];
    
    // Restaurar posición inicial (boca del cañón)
    p.x = 50;
    p.y = height - 40;
    
    // Recalcular velocidades iniciales con dispersión angular
    let launchAngle = angle + i * 5;
    p.vx = v0 * cos(launchAngle);
    p.vy = -v0 * sin(launchAngle);
    
    // Limpiar estela anterior
    p.trail = [];
    
    // Reactivar proyectil
    p.active = true;
  }
}

### CELDA 10: Bucle Principal (draw)
Se ejecuta 60 veces por segundo:
1. Limpiar pantalla (background)
2. Dibujar suelo y cañón (estáticos)
3. Actualizar proyectiles (Euler + colisiones)
4. Dibujar proyectiles (visuales)
5. Reiniciar cada 240 frames
6. Mostrar información en pantalla (HUD)

In [11]:
// ============================================================
// FUNCIÓN PRINCIPAL DRAW (BUCLE DE ANIMACIÓN)
// Se ejecuta continuamente a aproximadamente 60 FPS
// 5.11 Celda 10: Función draw Principal
// ============================================================

function draw() {
  
  // Fondo oscuro (color azul marino profundo)
  background('#1a1a2e');
  
  // Verificación de seguridad: si no hay proyectiles, no continuar
  if (!projectiles || projectiles.length === 0) return;
  
  // Dibujar elementos estáticos
  drawGround();
  drawCannon();
  
  // Actualizar y dibujar proyectiles dinámicos
  updateProjectiles();
  drawProjectiles();
  
  // Reiniciar automáticamente cada 240 frames (4 segundos aprox)
  if (frameCount % 240 === 0) {
    resetProjectiles();
  }
  
  // Información en pantalla (HUD)
  fill(0, 0, 100);
  noStroke();
  textSize(14);
  
  // Mostrar método numérico usado (CAMBIADO A EULER)
  text("Método: Euler Explícito (Forward Euler)", 10, 20);
  text("Orden de precisión: O(Δt)", 10, 35);
  
  // Parámetros físicos
  text("v0 = " + v0 + " px/frame", 10, 55);
  text("θ = " + angle + "°", 10, 70);
  text("g = " + g + " px/frame²", 10, 85);
  
  // Parámetros de simulación
  text("Rebote: " + (restitution * 100).toFixed(0) + "%", 10, 105);
  text("Δt = " + dt + " frame", 10, 120);
  text("Proyectiles: " + numProjectiles, 10, 135);
}

### CELDA 11: Renderizado
%show<br>
Instrucción mágica de JupyterLite para mostrar el canvas de p5.js.

In [12]:
%show

### Diferencias clave con RK4

| Aspecto                     | Euler (este código)                    | RK4 (original)           |
| --------------------------- | -------------------------------------- | ------------------------ |
| **Evaluaciones**            | 1 por paso                             | 4 por paso               |
| **Fuentes de error**        | Solo pendiente inicial                 | Promedio de 4 pendientes |
| **Precisión trayectoria**   | Baja (se desvía)                       | Alta (casi exacta)       |
| **Velocidad computacional** | ~4x más rápido                         | Más lento pero preciso   |
| **Visual**                  | Trayectorias "temblorosas" o desviadas | Trayectorias suaves      |

Para apreciar la diferencia, prueba aumentar g a 1.0 o v0 a 30: Euler mostrará inestabilidad visible, mientras que RK4 seguiría estable.
